In [1]:
%matplotlib inline
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'
os.environ['MKL_NUM_THREADS'] = '32'
os.environ['GOTO_NUM_THREADS'] = '32'
os.environ['OMP_NUM_THREADS'] = '32'
os.environ['openmp'] = 'True'


import matplotlib.pyplot as plt
import matplotlib.colors as colors
import librosa
import numpy as np
import glob

import keras
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import Dense, Activation

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
descriptors_folder = 'Descriptores-cuts/'
npys = []
for file in glob.glob(descriptors_folder+'*.npy'):
    if not '_st' in file:
        x = np.load(file)
        npys.append(x)
X = np.hstack(npys).T
sounds = []
for filename in glob.glob('Cuts/*.wav'):
    x,fs = librosa.load(filename, sr = None)
    sounds.append(x[:22016])
    
S = np.vstack(sounds)
F = np.fft.fft(S)
F = F[:int(S.shape[1]/2)]

In [ ]:
model = Sequential()
model.add(Dense(F.shape[1], input_dim=X.shape[1]),)
model.add(Activation('relu'))

sgd = SGD(lr=0.05, clipnorm=1.)

model.compile(loss='mean_squared_error', optimizer=sgd)

history = model.fit(X, abs(F), batch_size=4, epochs=5000, verbose=0)

/usr/local/lib/python2.7/dist-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


In [ ]:
plt.plot(history.history['loss'])